In [8]:
import os
import pandas as pd
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [2]:
datapath = os.path.join('./test_Indosum (1).csv')

data_test = pd.read_csv(datapath, encoding='latin-1')
data_test.head()

,paragraphs,summary
0,Jakarta CNN Indonesia - - Bayern Munich menja...,Bayern Munich menjadi klub raksasa Eropa teran...
1,Merdeka.com - Catatan tak pernah kalah yang di...,Catatan tak pernah kalah yang dirangkum oleh P...
2,Instagram Stories adalah candu baru buat merek...,Kita mungkom telah banyak melihat ada teman ya...
3,Pemerintah bersama dengan Kepolisian RI TNI ...,Pemerintah bersama dengan Kepolisian RI TNI ...
4,Mantan Bupati Buton Samsu Umar Abdul Samiun d...,Mantan Bupati Buton Samsu Umar Abdul Samiun d...


In [11]:
import pandas as pd
from torch.utils.data import Dataset

class T5Dataset(Dataset):
    def __init__(self, path, tokenizer):
        self.df = pd.read_csv(path, on_bad_lines='skip')
        self.summary = self.df["summary"]
        self.paragraphs = self.df["paragraphs"]
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.paragraphs)

    def __getitem__(self, index):
        paragraphs = str(self.paragraphs[index])
        summary = str(self.summary[index])
        pad = self.tokenizer.pad_token
        eos = self.tokenizer.eos_token
        
        # Encode input (paragraf) dengan prefix "ringkasan: "
        encoding_paragraphs = self.tokenizer.encode_plus(
            "ringkasan: " + paragraphs,
            return_token_type_ids=False,
            return_attention_mask=True,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors='pt'
        )

        # Encode output (ringkasan)
        encoding_summary = self.tokenizer.encode(
            pad + summary,
            add_special_tokens=False,
            return_token_type_ids=False,
            max_length=150,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'sentence_text': paragraphs,
            'summary_text': summary,
            'input_ids': encoding_paragraphs['input_ids'].flatten(),
            'attention_mask': encoding_paragraphs['attention_mask'].flatten(),
            'lm_labels': encoding_summary.flatten(),
        }

In [12]:
train_set = T5Dataset("./train_Indosum (1).csv",tokenizer)
train_loader = DataLoader(train_set, batch_size = 8,shuffle = True)
val_set = T5Dataset("./test_Indosum (1).csv",tokenizer)
val_loader = DataLoader(val_set, batch_size = 8,shuffle = True)

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

save_path = "./model_summarization/"

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)

print("Model and tokenizer successfully loaded!")

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Model and tokenizer successfully loaded!


In [14]:
with torch.no_grad():
    data = next(iter(val_loader))
    sentence_text, summary_text, input_ids, attention_mask, lm_labels = (
        data["sentence_text"],
        data["summary_text"],
        data["input_ids"],
        data["attention_mask"],
        data["lm_labels"],
    )

    # Generate summary
    generated = model.generate(
    input_ids=input_ids,
            min_length=20,
            max_length=80,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95,
    )

    # Decode generated output
    decoded_summary = tokenizer.decode(
        generated[0],
        skip_special_tokens=True,  # Hilangkan token spesial seperti <pad>, <sos>, dll.
        clean_up_tokenization_spaces=True,  # Hapus spasi tambahan
    )

    # Print results
    print("Full text:")
    print(sentence_text[0])  # Pastikan sentence_text berupa string
    print("\nReference summary:")
    print(summary_text[0])  # Pastikan summary_text berupa string
    print("\nGenerated summary:")
    print(decoded_summary)  # Hasil yang telah dibersihkan

Full text:
Lima puluh warga penggusuran kolong flyover jalan tol Pluit - Tomang  dekat RPTRA Kalijodo  siang ini masih bertahan di lokasi . Padahal warga yang selama ini tinggal di rumah bedeng tersebut telah direlokasi ke Rusun Marunda dan Rusun Kapuk oleh Pemprov DKI Jakarta . Pantauan kumparan ( kumparan.com )  warga secara bergerombol berteduh di pojok salah satu sisi bawah jembatan jalan tol . Nampak perabotan rumah tangga seperti kasur  kursi  kompor  dan perabotan rumah tangga berserakan di bawah flyover .  Baca juga : Warga Kalijodo : Katanya Gusur Abis Lebaran ?  Menurut salah satu warga bernama Leo  warga yang masih bertahan di bawah flyover tersebut merupakan warga ber - KTP DKI Jakarta yang menolak dipindahkan ke Rusun Marunda maupun Rusun Kapuk . Dia beralasan  kebebasan dalam bertempat tinggal dirampas oleh pemerintah .  Baca juga : Korban Penggusuran Kalijodo Akan Dipindah ke Rusun Marunda dan Kapuk  " Warga yang saat ini ada di sini ( bertahan di bawah kolong jembatan )